# Proyecto Final - Accidentes en USA

En este proyecto se va a utilizar un DataSet de accidentes en US y se va a buscar analizar la gravedad de los mismos y graficar los resultados

Debido a que el dataset es demasiado pesado no se va a subir al repositorio de git pero lo pueden descargar de este link:
[Dataset](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents?select=US_Accidents_March23.csv)

